In [1]:
import numpy as np
import torch
from hmmlearn.hmm import CategoricalHMM
import torch.nn.functional as F

In [4]:
def generate_starting_probabilities(states):
    return np.full(states, 1/states)

def generate_transition_matrix(states, stay_prob=0.95):
    # Create an empty transition matrix
    transition_matrix = np.zeros((states, states))
    change_prob = 1 - stay_prob

    # Set the probabilities for the first and last states
    transition_matrix[0, 0] = stay_prob
    transition_matrix[0, 1] = change_prob
    transition_matrix[states-1, states-1] = stay_prob
    transition_matrix[states-1, states-2] = change_prob

    # Set the probabilities for the middle states
    for i in range(1, states-1):
        transition_matrix[i, i] = stay_prob
        transition_matrix[i, i-1] = change_prob/2
        transition_matrix[i, i+1] = change_prob/2

    # Normalize the rows of the transition matrix
    row_sums = transition_matrix.sum(axis=1)
    transition_matrix = transition_matrix / row_sums[:, np.newaxis]

    return transition_matrix

def generate_emission_probabilities(states, outputs):
    emission_probabilities = np.zeros((states, outputs))

    # Calculate the center of the distribution for each state
    centers = np.linspace(0, outputs - 1, num=states)

    for i in range(states):
        center = centers[i]
        for j in range(outputs):
            distance = np.abs(j - center)

            # Use a Gaussian-like distribution to assign probabilities
            emission_probabilities[i, j] = np.exp(-0.2 * (distance / (0.1 * (outputs - 1)))**2)

    # Normalize probabilities for each state to sum to 1
    emission_probabilities /= emission_probabilities.sum(axis=1, keepdims=True)

    return emission_probabilities

def generate_hmm_data(start_probabilities, transition_matrix, emission_probabilities, num_seq, seq_len, outputs):
    """
    Generates sequences and hidden states from an HMM and formats them for machine learning tasks.

    Parameters:
        start_probabilities (array): Initial state probabilities.
        transition_matrix (array): State transition probabilities.
        emission_probabilities (array): Observation emission probabilities.
        num_seq (int): Number of sequences to generate.
        seq_len (int): Length of each sequence.
        outputs (int): Number of possible output symbols (for one-hot encoding).

    Returns:
        torch.Tensor: One-hot encoded observation sequences. (Shape: num_seq, seq_len, outputs)
        np.ndarray: Hidden state sequences.
    """
    # Initialize HMM model
    model = CategoricalHMM(n_components=len(start_probabilities))
    model.startprob_ = start_probabilities
    model.transmat_ = transition_matrix
    model.emissionprob_ = emission_probabilities

    # Generate sequences
    sampled_sequences = np.zeros((num_seq, seq_len))
    sampled_states = np.zeros((num_seq, seq_len))
    for i in range(num_seq):
        observations, hidden_states = model.sample(seq_len)
        sampled_sequences[i] = observations.reshape((seq_len))
        sampled_states[i] = hidden_states

    # One-hot encode the observation sequences
    one_hot_sequences = F.one_hot(torch.tensor(sampled_sequences).long(), num_classes=outputs)

    return one_hot_sequences, sampled_states

In [6]:
start, trans, emiss = np.array([1,0]), generate_transition_matrix(2, 0.95), generate_emission_probabilities(2, 3)
num_seq, seq_len, outputs = 5000, 100, 3
seq, states = generate_hmm_data(start, trans, emiss, num_seq, seq_len, outputs)

In [7]:
count = 0
for i in range(num_seq):
    if states[i][1] == 1:
        count += 1
print(count/num_seq)

0.0454


In [4]:
def build_hmm_emissions(num_states):
    """
    Returns a list of emission probability vectors for an HMM
    whose states linearly interpolate from [0.99, 0.01, 0]
    to [0, 0.01, 0.99].

    Mathematically, we define an interpolation factor α_i:

        α_i = (i - 1) / (num_states - 1),   for i = 1, 2, ..., num_states

    This ensures:
      - At i = 1 (first state), α_1 = 0, so the probabilities are [0.99, 0.01, 0].
      - At i = num_states (last state), α_M = 1, so the probabilities are [0, 0.01, 0.99].
      - For 1 < i < M, α_i evenly interpolates between 0 and 1.

    The emission probabilities for state i are then computed as:

        p1(i) = 0.99 * (1 - α_i)    # Probability for output 1
        p2(i) = 0.01                # Probability for output 2 (constant)
        p3(i) = 0.99 * α_i          # Probability for output 3

    Since p1(i) + p2(i) + p3(i) = 1 for all i, the probabilities are correctly normalized.

    Arguments:
    num_states -- integer, the total number of hidden states M

    Returns:
    emissions  -- list of lists, each being [p1, p2, p3]
    """
    emissions = []
    for i in range(1, num_states + 1):
        alpha_i = (i - 1) / (num_states - 1)  # Interpolation factor
        p1 = 0.99 * (1 - alpha_i)    # Probability for output 1
        p2 = 0.01                    # Probability for output 2 (constant)
        p3 = 0.99 * alpha_i          # Probability for output 3
        emissions.append([p1, p2, p3])
    
    return emissions


# Example usage:
# Suppose we want 5 states
M = 10
emission_probs = build_hmm_emissions(M)
for state_index, probs in enumerate(emission_probs, start=1):
    print(f"State {state_index}: {probs}")


State 1: [0.99, 0.01, 0.0]
State 2: [0.8799999999999999, 0.01, 0.10999999999999999]
State 3: [0.77, 0.01, 0.21999999999999997]
State 4: [0.66, 0.01, 0.32999999999999996]
State 5: [0.55, 0.01, 0.43999999999999995]
State 6: [0.43999999999999995, 0.01, 0.55]
State 7: [0.33, 0.01, 0.6599999999999999]
State 8: [0.21999999999999997, 0.01, 0.77]
State 9: [0.11000000000000004, 0.01, 0.8799999999999999]
State 10: [0.0, 0.01, 0.99]
